In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

spark=SparkSession.builder.appName('desafio-python3').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/06 19:30:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/06 19:30:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [3]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

In [4]:
dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/T005T.parquet")

#### Filter Mandante - Idioma Pais

In [5]:
df_metadata_facturas = dataframes_dict['metadata_de_facturas'].where(col('Mandante') == '400')

df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].where(col('Mandante') == '400')

df_materiales = dataframes_dict['materiales'].where(col('Mandante') == '400')

df_paises = dataframes_dict['paises'].where(col('Mandante') == '400').where(col('Idioma') == 'S')

#### Select tables

In [6]:
df_metadata_facturas = df_metadata_facturas.select('Factura', 'Clase_factura', 'Ejercicio', 'Per_contable').where(col('Factura').isNotNull())


In [7]:
df_facturas_por_cliente_destinatario = df_facturas_por_cliente.select('Factura','Posicion_de_Factura','Pais', 'Nombre', 'Numero_de_Cliente', 'Cliente_Destinatario_de_Mercancia')\
                                                .where(col('Cliente_Destinatario_de_Mercancia').isNotNull())\
                                                .withColumnRenamed('Pais', 'Pais_cliente')

In [8]:
df_materiales = df_materiales.select('Factura', 'Posicion', 'Material', 'Unidad_de_medida_Volumen_Destino', 'Volumen')


In [9]:
df_cliente_solicitante_factura = df_facturas_por_cliente.select('Factura', 'Cliente_solicitante').where(col('Cliente_solicitante').isNotNull())

In [10]:
df_paises = df_paises.select('Idioma', 'Pais' , 'Pais_descr')

#### Join tables

In [11]:
def set_posicion_destinatario_mercancia(dataframes_dict):

    df1 = df_materiales
    df2 = df_facturas_por_cliente_destinatario

    join_cond = [(df1.Factura == df2.Factura) & (df1.Posicion == df2.Posicion_de_Factura)]

    df_result = df1.join(df2, join_cond, 'left')\
                             .drop(df2.Posicion_de_Factura)\
                             .drop(df2.Factura)

    return{'add_posicion_destinatario_mercancia' : df_result}


def set_clase_factura(dataframes_dict):

    df = dataframes_dict['add_posicion_destinatario_mercancia']

    df_result = df.join(df_metadata_facturas, 'Factura', 'left')

    return{'add_clase_factura' : df_result}


def set_cliente_solitante(dataframes_dict):

    df = dataframes_dict['add_clase_factura']

    df_result = df.join(df_cliente_solicitante_factura, 'Factura' , 'left')

    return{'add_cliente_solitante' : df_result}


def set_paises_desc(dataframes_dict):

    df = dataframes_dict['add_cliente_solitante']

    df_result = df.join(df_paises,df.Pais_cliente == df_paises.Pais, 'left')\
                                      .drop(df_paises.Pais)

    return {'add_paises_descripcion': df_result}

In [12]:
def select_columns(dataframes_dict):

    df_final = dataframes_dict['add_paises_descripcion']\
                    .select('Factura', 'Clase_factura', 'Posicion',
                    'Material', 'Volumen', 'Unidad_de_medida_Volumen_Destino',
                    'Per_contable', 'Ejercicio','Nombre', 'Numero_de_Cliente','Cliente_solicitante', 'Cliente_Destinatario_de_Mercancia',
                    'Pais_cliente', 'Pais_descr')\
                    .withColumnRenamed('Nombre', 'Nombre_cliente')

    return{'add_select_columns' : df_final}

In [13]:
def main(dataframes_dict):

    functions_list = [
        set_posicion_destinatario_mercancia,
        set_clase_factura,
        set_cliente_solitante,
        set_paises_desc,
        select_columns
        # Otras transformaciones
    ]

    for function in functions_list:
        print(function)
        result_element = function(dataframes_dict)
        dataframes_dict.update(result_element)

    return dataframes_dict['add_select_columns'] # Modificar al último dataframe retornado

In [14]:
test = main(dataframes_dict)

<function set_posicion_destinatario_mercancia at 0x115363250>
<function set_clase_factura at 0x1153631c0>
<function set_cliente_solitante at 0x115362e60>
<function set_paises_desc at 0x115362dd0>
<function select_columns at 0x115362b00>


In [15]:
dataframes_dict['add_select_columns']

Factura,Clase_factura,Posicion,Material,Volumen,Unidad_de_medida_Volumen_Destino,Per_contable,Ejercicio,Nombre_cliente,Numero_de_Cliente,Cliente_solicitante,Cliente_Destinatario_de_Mercancia,Pais_cliente,Pais_descr
0150190076,YN01,10,000000000010010108,5600.0,M3,12,2020,Reciclaje de Metales Ingeni...,164952,164952,164952,CL,CHILE
0150113050,YPVN,60,M100PLY000122,0.0,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,10,M100PLY000129,0.0,M3,12,2020,null,169397,17309,169397,null,null
0150113050,YPVN,61,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,62,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,63,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,64,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,65,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,66,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null
0150113050,YPVN,67,M100PLY000122,2.8934280639431615,M3,12,2020,null,null,17309,null,null,null


In [16]:
print('materiales_original :' , dataframes_dict['materiales'].agg(sum(col('Volumen'))))
print('add_select_columns :' ,  dataframes_dict['add_select_columns'].agg(sum(col('Volumen'))))



materiales_original : +-----------------+
|     sum(Volumen)|
+-----------------+
|899471.0868944847|
+-----------------+

add_select_columns : +-----------------+
|     sum(Volumen)|
+-----------------+
|899471.0868944847|
+-----------------+



In [17]:
#### Cantidad de facturas por tipo cliente
dataframes_dict['facturas_por_cliente'].groupBy('Tipo_de_Cliente').agg(count('Factura').alias('Cantidad_Facturas')).orderBy(col('Cantidad_Facturas'))

Tipo_de_Cliente,Cantidad_Facturas
NF,17
SP,196
AV,530
EN,671
SE,1306
VD,2423
RG,2798
VE,2798
RE,2798
AG,2798


In [32]:
dataframes_dict['add_select_columns'].groupBy('Pais_cliente', 'Cliente_Destinatario_de_Mercancia').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show(10, False)

+------------+---------------------------------+------------------+
|Pais_cliente|Cliente_Destinatario_de_Mercancia|Volumen_total     |
+------------+---------------------------------+------------------+
|null        |169539                           |445009.6594002447 |
|null        |null                             |225016.99347758506|
|CL          |164952                           |112860.0          |
|CL          |243596                           |112000.0          |
|DK          |68848                            |512.6345536023433 |
|AE          |286394                           |400.4             |
|null        |169454                           |286.34562670005977|
|MX          |119088                           |208.32682060390763|
|GB          |52407                            |206.51459230313796|
|ES          |66146                            |188.65469947559498|
+------------+---------------------------------+------------------+
only showing top 10 rows



In [35]:
dataframes_dict['add_select_columns'].groupBy('Cliente_solicitante').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show(5, False)

+-------------------+------------------+
|Cliente_solicitante|Volumen_total     |
+-------------------+------------------+
|15847              |446360.3014697138 |
|243596             |122000.0          |
|164952             |118530.0          |
|C6102              |10610.667462500058|
|C6108              |8836.122548012454 |
+-------------------+------------------+
only showing top 5 rows



In [38]:
dataframes_dict['add_select_columns'].groupBy('Clase_factura').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show(9999999, False)

+-------------+------------------+
|Clase_factura|Volumen_total     |
+-------------+------------------+
|YNDN         |445194.87803089235|
|YN01         |240556.0          |
|YPVE         |103025.39805597173|
|ZINM         |55830.77846639777 |
|YPVN         |47180.698345900484|
|YSDE         |4537.500745486273 |
|YSCE         |1578.4242525546445|
|YSDN         |654.7385285745437 |
|YSCN         |351.38799145984177|
|YPBN         |295.21406943976496|
|YNDE         |266.0684079870466 |
|ZB03         |0.0               |
|ZB13         |0.0               |
|ZB01         |0.0               |
+-------------+------------------+



#### Falta hacer el parquet para las consultas
#### Falta dejar solo las consultas en notebook lo demas va en el .py